In [1]:
!pip install --quiet altair
!pip install --quiet tabulate


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
import altair as alt
import pandas as pd

from tabulate import tabulate

In [3]:
click_df = pd.read_parquet("output/clicks.parquet", columns=["query", "title", "abstract", "url_md5", "position", "click"])
sample_df = pd.read_parquet("output/samples.parquet", columns=["query", "title", "abstract", "position", "click"])
annotation_df = pd.read_parquet("output/annotations.parquet")

## Utils

In [4]:
def print_table(stats):
    stats_table = [[k, f"{v:,}"] for k, v in stats.items()]
    print(tabulate(stats_table, headers=["Statistic", "Value"], tablefmt="pretty"))

def match(df1, df2, on, left_columns=[], right_columns=[], unique=False):
    
    if unique:
        df1 = df1[on + left_columns].drop_duplicates(on)
        df2 = df2[on + right_columns].drop_duplicates(on)

    return df1[on + left_columns].merge(df2[on + right_columns], on=on)

def missing_text(df, column, missing_token="21429"):
    return (df[column] == missing_token).agg(["sum", "mean"]).round(decimals=4).to_dict()

def count_stats(df, groupby, unique = False):
    if unique:
        df = df.drop_duplicates(["query", "title", groupby])
    
    source = df.groupby([groupby]).agg(
        documents=("title", "count"),
    ).reset_index()

    source["perc_documents"] = (source.documents / source.documents.sum()).round(4)
    return source

## Part I: Expert annotations

In [5]:
print(f"Unique queries (by text): {annotation_df['query'].nunique():,}")
print(f"Unique queries (by id): {annotation_df['query_id'].nunique():,}")
print(f"Query-doc pairs: {len(annotation_df):,}")
print(f"Unique query/title comb.: {len(annotation_df[['query_id', 'title']].drop_duplicates()):,}")
print()
print(f"Unique titles per query:\n{annotation_df.groupby('query')['title'].nunique().describe(percentiles=[0.5])}\n")

Unique queries (by text): 7,008
Unique queries (by id): 5,201
Query-doc pairs: 397,572
Unique query/title comb.: 368,792

Unique titles per query:
count    7008.000000
mean       52.628282
std         9.313274
min         1.000000
50%        54.000000
max       109.000000
Name: title, dtype: float64



In [6]:
print(f"Missing query: {missing_text(annotation_df, column='query')}")
print(f"Missing title: {missing_text(annotation_df, column='title')}")
print(f"Missing abstract: {missing_text(annotation_df, column='abstract')}")

Missing query: {'sum': 0.0, 'mean': 0.0}
Missing title: {'sum': 0.0, 'mean': 0.0}
Missing abstract: {'sum': 0.0, 'mean': 0.0}


## Part II: Click dataset

In [7]:
print(f"Unique queries (by text): {click_df['query'].nunique():,}")
print(f"Query-doc pairs: {len(click_df):,}")
print(f"Unique query/title comb.: {len(click_df[['query', 'title']].drop_duplicates()):,}")
print()
print(f"Unique urls per query:\n{click_df.groupby('query')['url_md5'].nunique().describe(percentiles=[0.5])}")
print()
print(f"Unique titles per query:\n{click_df.groupby('query')['title'].nunique().describe(percentiles=[0.5])}")

Unique queries (by text): 3,366
Query-doc pairs: 59,891,217
Unique query/title comb.: 106,092

Unique urls per query:
count     3366.000000
mean       112.923648
std        947.676248
min          2.000000
50%         22.000000
max      28439.000000
Name: url_md5, dtype: float64

Unique titles per query:
count    3366.000000
mean       31.518717
std       116.702019
min         2.000000
50%        19.000000
max      4335.000000
Name: title, dtype: float64


In [8]:
print(f"Missing query in filtered subset: {missing_text(click_df, column='query')}, overall in Baidu-ULTR: {missing_text(sample_df, column='query')}")
print(f"Missing title in filtered subset: {missing_text(click_df, column='title')}, overall in Baidu-ULTR: {missing_text(sample_df, column='title')}")
print(f"Missing abstract: {missing_text(click_df, column='abstract')}, overall in Baidu-ULTR: {missing_text(sample_df, column='abstract')}")

Missing query in filtered subset: {'sum': 0.0, 'mean': 0.0}, overall in Baidu-ULTR: {'sum': 20.0, 'mean': 0.0}
Missing title in filtered subset: {'sum': 9744019.0, 'mean': 0.1627}, overall in Baidu-ULTR: {'sum': 816667.0, 'mean': 0.1302}
Missing abstract: {'sum': 56333874.0, 'mean': 0.9406}, overall in Baidu-ULTR: {'sum': 5318177.0, 'mean': 0.848}


## Part III: Match between clicks and annotations:

In [9]:
query_df = match(annotation_df, click_df, on=['query'], unique=True)

print(f"Queries in both datasets: {len(query_df):,}")
print(f"% of annotated queries in both datasets: % of all test queries: {len(query_df) / annotation_df['query'].nunique():.4f}")
print()
print(f"Query/title comb. in both datasets: {len(match(annotation_df, click_df, on=['query', 'title'])):,}")
print(f"Unique query/title comb. in both datasets: {len(match(annotation_df, click_df, on=['query', 'title'], unique=True)):,}")
print(f"% of query/title comb. of the test set occur in both datasets: {len(match(annotation_df, click_df, on=['query', 'title'], unique=True)) / len(annotation_df[['query', 'title']].drop_duplicates()):.4f}")
print()
print(f"Query/title/abstract comb. in both datasets: {len(match(annotation_df, click_df, on=['query', 'title', 'abstract'])):,}")

Queries in both datasets: 3,366
% of annotated queries in both datasets: % of all test queries: 0.4803

Query/title comb. in both datasets: 51,746,434
Unique query/title comb. in both datasets: 8,672
% of query/title comb. of the test set occur in both datasets: 0.0235

Query/title/abstract comb. in both datasets: 0


## Part IV: Focusing on query/title matches

In [10]:
match_df = match(annotation_df, click_df, on=['query', 'title'], left_columns=["frequency_bucket", "label"], right_columns=["position", "click"])

print(f"Queries that have at least one matching document (by title): {match_df['query'].nunique():,}")
print(f"% of queries that occur in the click dataset AND have matching documents: {match_df['query'].nunique() / annotation_df['query'].nunique():.4f}")
print()
print(f"Unique documents per query:\n{match_df.groupby('query')['title'].nunique().describe(percentiles=[0.5, 0.9]).apply(lambda x: format(x, '.2f'))}")
print()
print(f"Documents per query:\n{match_df.groupby('query').size().describe(percentiles=[0.5, 0.9]).apply(lambda x: format(x, '.2f'))}")

Queries that have at least one matching document (by title): 2,744
% of queries that occur in the click dataset AND have matching documents: 0.3916

Unique documents per query:
count    2744.00
mean        3.16
std         2.08
min         1.00
50%         3.00
90%         6.00
max        19.00
Name: title, dtype: object

Documents per query:
count        2744.00
mean        18858.03
std        621974.72
min             1.00
50%            67.00
90%          5966.60
max      32394427.00
dtype: object


In [11]:
source = match_df.groupby("query").agg(total_documents=("title", "count"), unique_documents=("title", "nunique")).reset_index()

base = alt.Chart(source).transform_calculate(
    log_total_documents = 'log(datum.total_documents)/log(10)'
)

base.mark_bar().encode(
    x=alt.X("unique_documents:O", title="# of unique docs per query"),
    y=alt.Y("count(query):Q", title="# of queries"),
    tooltip=["unique_documents:O", "count(query):Q"],
) | base.mark_bar().encode(
    x=alt.X("log_total_documents:Q", title="# of total docs per query (log scale)").bin().axis(labelExpr='format(pow(10, datum.value), ",")', labelAngle=-45, labelOverlap=False),
    y=alt.Y("count(query):Q", title="# of queries"),
    tooltip=alt.Tooltip(["count(query):Q"], bin=True),
)

alt.HConcatChart(...)

#### Relevance distribution shift

In [12]:
alt.Chart(
    count_stats(annotation_df, groupby="label", unique=True),
    title="Expert Annotations",
    width=400,
    height=150,
).mark_bar().encode(
    x=alt.X("label:O", title="Relevance label").axis(labelAngle=0),
    y=alt.Y("perc_documents", title="% of (unique) query/title pairs").axis(format="%"),
    tooltip=["perc_documents"],
) & alt.Chart(
    count_stats(match_df, groupby="label", unique=True),
    title="Expert Annotations + Clicks",
    width=400,
    height=150,
).mark_bar().encode(
    x=alt.X("label:O", title="Relevance label").axis(labelAngle=0),
    y=alt.Y("perc_documents", title="% of (unique) query/title pairs").axis(format="%"),
    tooltip=["perc_documents"],
)

alt.VConcatChart(...)

#### Query popularity shift

In [13]:
alt.Chart(
    count_stats(annotation_df, groupby="frequency_bucket", unique=True),
    title="Expert Annotations",
    width=600,
    height=150,
).mark_bar().encode(
    x=alt.X("frequency_bucket:O", title="Query frequency (high to low)").axis(labelAngle=0),
    y=alt.Y("perc_documents", title="% of (unique) query/title pairs").axis(format="%"),
    tooltip=["perc_documents"],
) & alt.Chart(
    count_stats(match_df, groupby="frequency_bucket", unique=True),
    title="Expert Annotations + Clicks",
    width=600,
    height=150,
).mark_bar().encode(
    x=alt.X("frequency_bucket:O", title="Query frequency (high to low)").axis(labelAngle=0),
    y=alt.Y("perc_documents", title="% of (unique) query/title pairs").axis(format="%"),
    tooltip=["perc_documents"],
)

alt.VConcatChart(...)

#### Doc positions shift

In [14]:
alt.Chart(
    count_stats(sample_df, groupby="position", unique=True),
    title="Clicks",
    width=600,
    height=150,
).mark_bar().encode(
    x=alt.X("position:O", title="Query frequency (high to low)").axis(labelAngle=0),
    y=alt.Y("perc_documents", title="% of (unique) query/title pairs").axis(format="%"),
    tooltip=["perc_documents"],
) & alt.Chart(
    count_stats(match_df, groupby="position", unique=True),
    title="Expert Annotations + Clicks",
    width=600,
    height=150,
).mark_bar().encode(
    x=alt.X("position:O", title="Query frequency (high to low)").axis(labelAngle=0),
    y=alt.Y("perc_documents", title="% of (unique) query/title pairs").axis(format="%"),
    tooltip=["perc_documents"],
)

alt.VConcatChart(...)

## Part V: Focusing on queries that have matches in consecutive positions

In [15]:
def longest_consecutive_subsequence(positions):
    positions = sorted(positions)
    longest_streak = []
    current_streak = [positions[0]]
    
    for i in range(1, len(positions)):
        if positions[i] == positions[i - 1] + 1:
            current_streak.append(positions[i])
        else:
            if len(current_streak) > len(longest_streak):
                longest_streak = current_streak
            current_streak = [positions[i]]
    
    if len(current_streak) > len(longest_streak):
        longest_streak = current_streak
        
    return longest_streak

position_df = match_df.drop_duplicates(["query", "position"]).groupby('query').agg(positions=("position", longest_consecutive_subsequence)).reset_index()
position_df["min_position"] = position_df["positions"].map(lambda x: x[0])
position_df["max_position"] = position_df["positions"].map(lambda x: x[-1])
position_df["consecutive_positions"] = position_df["positions"].map(len)
position_df.head()

,query,positions,min_position,max_position,consecutive_positions
0,10016 2638 21326 21355 19701 5924 19701 18483 ...,"[7, 8, 9, 10]",7,10,4
1,10016 2638 9066 15533 9763 8207 16556 19675 10...,"[7, 8]",7,8,2
2,10048 8815 12496 19165 13933 9771 13933 6145 6...,[2],2,2,1
3,10059 12307 7897 9102 241 11676,"[4, 5, 6, 7, 8]",4,8,5
4,10059 16260 3742 3973 15425 20812,"[3, 4, 5, 6, 7, 8, 9, 10, 11]",3,11,9


In [22]:
top_10_df = position_df[(position_df.min_position == 1) & (position_df.consecutive_positions >= 10)]
top_10_df = match_df[match_df["query"].isin(top_10_df["query"])]

print(f"Queries with at documents in at least three consecutive positions: {len(position_df[position_df.consecutive_positions >= 3]):,}")
print(f"Queries with consecutive top 3: {len(position_df[(position_df.min_position == 1) & (position_df.consecutive_positions >= 3)]):,}")
print(f"Queries with consecutive top 10: {len(position_df[(position_df.min_position == 1) & (position_df.consecutive_positions >= 10)]):,}, with {len(top_10_df):,} query/document pairs")

Queries with at documents in at least three consecutive positions: 1,813
Queries with consecutive top 3: 442
Queries with consecutive top 10: 307, with 48,281,659 query/document pairs


In [23]:
alt.Chart(
    position_df,
    title="Clicks",
    width=600,
    height=150,
).mark_bar().encode(
    x=alt.X("consecutive_positions:O", title="# of documents in consecutive positions").axis(labelAngle=0),
    y=alt.Y("count(query)", title="# of queries"),
)

alt.Chart(...)

In [27]:
alt.Chart(
    count_stats(top_10_df, groupby="frequency_bucket", unique=True),
    title="Expert Annotations + Clicks",
    width=600,
    height=150,
).mark_bar().encode(
    x=alt.X("frequency_bucket:O", title="Query frequency (high to low)").axis(labelAngle=0),
    y=alt.Y("perc_documents", title="% of (unique) query/title pairs").axis(format="%"),
    tooltip=["perc_documents"],
) & alt.Chart(
    count_stats(top_10_df, groupby="label", unique=True),
    title="Expert Annotations + Clicks",
    width=400,
    height=150,
).mark_bar().encode(
    x=alt.X("label:O", title="Relevance label").axis(labelAngle=0),
    y=alt.Y("perc_documents", title="% of (unique) query/title pairs").axis(format="%"),
    tooltip=["perc_documents"],
)

alt.VConcatChart(...)

## Part VI: User-annotator agreement on top 10 subset
Trust bias?!?! and cascade?

In [132]:
df = top_10_df.sort_values(["query", "position", "title"]).groupby(["query", "position", "title"]).agg(
    ctr=("click", "mean"),
    clicks=("click", "sum"),
    impressions=("click", "count"),
    label=("label", "max"),
).reset_index()

df.head()

(alt.Chart(df, title="Avg. CTR per position/expert label", width=250).mark_rect().encode(
    x="label:O",
    y="position:O",
    color="mean(ctr)",
    tooltip=["mean(ctr)", "sum(impressions)", "count(title)"],
) | alt.Chart(df, title="# of clicks per position/expert label", width=250).mark_rect().encode(
    x="label:O",
    y="position:O",
    color=alt.Color("sum(clicks)"),
) | alt.Chart(df, title="# of impressions per position/expert label", width=250).mark_rect().encode(
    x="label:O",
    y="position:O",
    color=alt.Color("sum(impressions)").scale(type="log"),
    tooltip=["sum(impressions)"]
)).resolve_scale(color="independent")

alt.HConcatChart(...)

In [137]:
alt.Chart(df, title="Avg. CTR per position/expert label", width=800).mark_line(point=True).encode(
    x="position:O",
    y="mean(ctr)",
    color="label:O",
    tooltip=["mean(ctr)", "sum(impressions)", "count(title)"],
) & alt.Chart(df, title="Avg. CTR per position/expert label", width=800).mark_bar().encode(
    x=alt.X("position:O"),
    xOffset="label:O",
    y=alt.Y("sum(impressions)").scale(type="log"),
    color="label:O",
    tooltip=["mean(ctr)", "sum(impressions)", "count(title)"],
) 

alt.VConcatChart(...)

In [123]:
df = match_df.sort_values(["query", "position", "title"]).groupby(["query", "position", "title"]).agg(
    ctr=("click", "mean"),
    clicks=("click", "sum"),
    impressions=("click", "count"),
    label=("label", "max"),
).reset_index()

df.head()

(alt.Chart(df, title="Avg. CTR per position/expert label", width=250).mark_rect().encode(
    x="label:O",
    y="position:O",
    color="mean(ctr)",
    tooltip=["mean(ctr)", "sum(impressions)", "count(title)"],
) | alt.Chart(df, title="# of clicks per position/expert label", width=250).mark_rect().encode(
    x="label:O",
    y="position:O",
    color=alt.Color("sum(clicks)"),
) | alt.Chart(df, title="# of impressions per position/expert label", width=250).mark_rect().encode(
    x="label:O",
    y="position:O",
    color=alt.Color("sum(impressions)").scale(type="log"),
    tooltip=["sum(impressions)"]
)).resolve_scale(color="independent")

alt.HConcatChart(...)